In [11]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import *
import matplotlib.pyplot as plt

In [41]:
def gyh(data):
    s=(data-data.min())/(data.max()-data.min())
    return s

def ycl(path,label):
    data=np.loadtxt(path)

    x_train=[]
    x_test=[]
    
    for i in range(10):
        data_new=data[0+i*100:102400+i*100]
        data_new=data_new.reshape(50,2048)

        np.random.shuffle(data_new)
        train=gyh(data_new[:40,:])
        test=gyh(data_new[40:,:])
        x_train.append(train)
        x_test.append(test)
    
    x_train=np.array(x_train).reshape(-1,2048)
    x_test=np.array(x_test).reshape(-1,2048)
    y_train=np.array([label for i in range(0,400)])
    y_test=np.array([label for i in range(0,100)])
    
    return x_train,y_train,x_test,y_test
def stackkk(a,b,c,d,e,f,g,h):
    aa=np.vstack((a,e))
    bb=np.hstack((b,f))
    cc=np.vstack((c,g))
    dd=np.hstack((d,h))
    return aa,bb,cc,dd
def dataset():
    '''
    0:正常轴承
    1：内圈受损0.007
    2：滚珠受损0.007
    3：外圈受损0.007
    4: 内圈受损0.014
    5: 滚珠受损0.014
    6: 外圈受损0.014
    7: 内圈受损0.021
    8: 滚珠受损0.021
    9: 外圈受损0.021
    
    '''
    x_train_1=[]
    y_train_1=[]
    x_test_1=[]
    y_test_1=[]
    x_train0,y_train0,x_test0,y_test0=ycl('./12K_21_2/Normal.txt',0)

    x_train1,y_train1,x_test1,y_test1=ycl('./12K_21_2/IR07.txt',1)
    x_train2,y_train2,x_test2,y_test2=ycl('./12K_21_2/B07.txt',2) 
    x_train3,y_train3,x_test3,y_test3=ycl('./12K_21_2/OR07.txt',3)
    
    x_train4,y_train4,x_test4,y_test4=ycl('./12K_21_2/IR14.txt',4)
    x_train5,y_train5,x_test5,y_test5=ycl('./12K_21_2/B14.txt',5) 
    x_train6,y_train6,x_test6,y_test6=ycl('./12K_21_2/OR14.txt',6)
    
    x_train7,y_train7,x_test7,y_test7=ycl('./12K_21_2/IR21.txt',7)
    x_train8,y_train8,x_test8,y_test8=ycl('./12K_21_2/B21.txt',8) 
    x_train9,y_train9,x_test9,y_test9=ycl('./12K_21_2/OR21.txt',9)
    
    x_train_1.extend([x_train0,x_train1,x_train2,x_train3,x_train4,x_train5,x_train6,x_train7,x_train8,x_train9])
    y_train_1.extend([y_train0,y_train1,y_train2,y_train3,y_train4,y_train5,y_train6,y_train7,y_train8,y_train9])
    x_test_1.extend([x_test0,x_test1,x_test2,x_test3,x_test4,x_test5,x_test6,x_test7,x_test8,x_test9])
    y_test_1.extend([y_test0,y_test1,y_test2,y_test3,y_test4,y_test5,y_test6,y_test7,y_test8,y_test9])

    
    a,b,c,d=x_train_1[0],y_train_1[0],x_test_1[0],y_test_1[0]
    for i in range(1,10,1):
        
        a,b,c,d=stackkk(a,b,c,d,x_train_1[i],y_train_1[i],x_test_1[i],y_test_1[i])

    
    return a,b,c,d

In [42]:
x_train,y_train,x_test,y_test=dataset()

In [45]:
x_train_=x_train.reshape(-1,2048,1)
x_test_=x_test.reshape(-1,2048,1)
y_train_=utils.to_categorical(y_train,num_classes=10)
y_test_=utils.to_categorical(y_test,num_classes=10)

In [46]:
print(x_train_.shape,y_train_.shape,x_test_.shape,y_test_.shape)

(4000, 2048, 1) (4000, 10) (1000, 2048, 1) (1000, 10)


In [ ]:
# 将数据与标签构造为一个元组
train_db=tf.data.Dataset.from_tensor_slices((x_train_,y_train_))
shuffle_db=train_db.shuffle(1000)# 打乱数据
train_data=shuffle_db.batch(100)# 分组

test_db=tf.data.Dataset.from_tensor_slices((x_test_,y_test_))
test_data=test_db.batch(100)

In [31]:
from tensorflow.keras import layers, Model, Sequential

# 18层，34层的网络残差模块
class BasicBlock(layers.Layer):
    expansion = 1

    def __init__(self, out_channel, strides=1, downsample=None, **kwargs):
        super(BasicBlock, self).__init__(**kwargs)
        self.conv1 = layers.Conv1D(out_channel, kernel_size=3, strides=strides,
                                   padding="SAME", use_bias=False)
        self.bn1 = layers.BatchNormalization(momentum=0.9, epsilon=1e-5)
        # -----------------------------------------
        self.conv2 = layers.Conv1D(out_channel, kernel_size=3, strides=1,
                                   padding="SAME", use_bias=False)
        self.bn2 = layers.BatchNormalization(momentum=0.9, epsilon=1e-5)
        # -----------------------------------------
        self.downsample = downsample
        self.relu = layers.ReLU()
        self.add = layers.Add()

    def call(self, inputs, training=False):
        identity = inputs
        if self.downsample is not None:
            identity = self.downsample(inputs)

        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = self.relu(x)

        x = self.conv2(x)
        x = self.bn2(x, training=training)

        x = self.add([identity, x])
        x = self.relu(x)

        return x

# 针对50，101，152层网络的残差模块
class Bottleneck(layers.Layer):
    expansion = 4

    def __init__(self, out_channel, strides=1, downsample=None, **kwargs):
        super(Bottleneck, self).__init__(**kwargs)
        self.conv1 = layers.Conv1D(out_channel, kernel_size=1, use_bias=False, name="conv1")
        self.bn1 = layers.BatchNormalization(momentum=0.9, epsilon=1e-5, name="conv1/BatchNorm")
        # -----------------------------------------
        self.conv2 = layers.Conv1D(out_channel, kernel_size=3, use_bias=False,
                                   strides=strides, padding="SAME", name="conv2")
        self.bn2 = layers.BatchNormalization(momentum=0.9, epsilon=1e-5, name="conv2/BatchNorm")
        # -----------------------------------------
        self.conv3 = layers.Conv1D(out_channel * self.expansion, kernel_size=1, use_bias=False, name="conv3")
        self.bn3 = layers.BatchNormalization(momentum=0.9, epsilon=1e-5, name="conv3/BatchNorm")
        # -----------------------------------------
        self.relu = layers.ReLU()
        self.downsample = downsample
        self.add = layers.Add()

    def call(self, inputs, training=False):
        identity = inputs
        if self.downsample is not None:
            identity = self.downsample(inputs)

        x = self.conv1(inputs)
        x = self.bn1(x, training=training)
        x = self.relu(x)

        x = self.conv2(x)
        x = self.bn2(x, training=training)
        x = self.relu(x)

        x = self.conv3(x)
        x = self.bn3(x, training=training)

        x = self.add([x, identity])
        x = self.relu(x)

        return x


def _make_layer(block, in_channel, channel, block_num, name, strides=1):
    downsample = None
    if strides != 1 or in_channel != channel * block.expansion:
        downsample = Sequential([
            layers.Conv1D(channel * block.expansion, kernel_size=1, strides=strides,
                          use_bias=False, name="conv1"),
            layers.BatchNormalization(momentum=0.9, epsilon=1.001e-5, name="BatchNorm")
        ], name="shortcut")

    layers_list = []
    layers_list.append(block(channel, downsample=downsample, strides=strides, name="unit_1"))

    for index in range(1, block_num):
        layers_list.append(block(channel, name="unit_" + str(index + 1)))

    return Sequential(layers_list, name=name)



def _resnet(block, blocks_num, im_len=2048, num_classes=1000, include_top=True):
    #block:残差模块，blocks_num,残差结构的个数
    # tensorflow中的tensor通道排序是NHWC
    # (None, 224, 224, 3)
    input_data = layers.Input(shape=(im_len,1), dtype="float32")
    x = layers.Conv1D(filters=64, kernel_size=7, strides=2,padding="SAME", use_bias=False, name="conv1")(input_data)
    x = layers.BatchNormalization(momentum=0.9, epsilon=1e-5, name="conv1/BatchNorm")(x)
    x = layers.ReLU()(x)
    x = layers.MaxPool1D(pool_size=3, strides=2, padding="SAME")(x)

    x = _make_layer(block, x.shape[-1], 64, blocks_num[0], name="block1")(x)
    x = _make_layer(block, x.shape[-1], 128, blocks_num[1], strides=2, name="block2")(x)
    x = _make_layer(block, x.shape[-1], 256, blocks_num[2], strides=2, name="block3")(x)
    x = _make_layer(block, x.shape[-1], 512, blocks_num[3], strides=2, name="block4")(x)

    if include_top:
        x = layers.GlobalAvgPool1D()(x)  # pool + flatten
        x = layers.Dense(num_classes, name="logits")(x)
        predict = layers.Softmax()(x)#最后的层数为1000
    else:
        predict = x

    model = Model(inputs=input_data, outputs=predict)

    return model

def resnet18(im_len=2048, num_classes=1000, include_top=True):
    return _resnet(BasicBlock, [2, 2, 2, 2], im_len, num_classes, include_top)

def resnet34(im_len=2048, num_classes=1000, include_top=True):
    return _resnet(BasicBlock, [3, 4, 6, 3], im_len, num_classes, include_top)


def resnet50(im_len=2048, num_classes=1000, include_top=True):
    return _resnet(Bottleneck, [3, 4, 6, 3], im_len,num_classes, include_top)


def resnet101(im_len=2048, num_classes=1000, include_top=True):
    return _resnet(Bottleneck, [3, 4, 23, 3], im_len, num_classes, include_top)



In [49]:
resnet=resnet18(include_top=False)
model=tf.keras.Sequential([
    resnet,
    layers.GlobalAveragePooling1D(),
    layers.Dropout(rate=0.5),
    layers.Dense(1024,activation='relu'),
    layers.Dropout(rate=0.5),
    layers.Dense(512,activation='relu'),
    layers.Dropout(rate=0.5),
    layers.Dense(10,activation='softmax')
    
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
functional_5 (Functional)    (None, 64, 512)           3853504   
_________________________________________________________________
global_average_pooling1d_2 ( (None, 512)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1024)              525312    
_________________________________________________________________
dropout_4 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)              

In [50]:
model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.categorical_crossentropy,
    metrics=[keras.metrics.categorical_accuracy]
)

In [52]:
history=model.fit(x_train_,y_train_,batch_size=100,epochs=10,validation_data=(x_test_,y_test_))

Epoch 1/10
40/40 [==============================] - 139s 3s/step - loss: 0.2391 - categorical_accuracy: 0.8963 - val_loss: 0.1595 - val_categorical_accuracy: 0.9370
Epoch 2/10
40/40 [==============================] - 141s 4s/step - loss: 0.1201 - categorical_accuracy: 0.9442 - val_loss: 0.0853 - val_categorical_accuracy: 0.9790
Epoch 3/10
40/40 [==============================] - 151s 4s/step - loss: 0.0448 - categorical_accuracy: 0.9890 - val_loss: 0.0527 - val_categorical_accuracy: 0.9890
Epoch 4/10
40/40 [==============================] - 164s 4s/step - loss: 0.0286 - categorical_accuracy: 0.9918 - val_loss: 0.0715 - val_categorical_accuracy: 0.9890
Epoch 5/10
40/40 [==============================] - 177s 4s/step - loss: 0.0121 - categorical_accuracy: 0.9962 - val_loss: 0.0649 - val_categorical_accuracy: 0.9900
Epoch 6/10
40/40 [==============================] - 173s 4s/step - loss: 0.0116 - categorical_accuracy: 0.9970 - val_loss: 0.0304 - val_categorical_accuracy: 0.9900
Epoch 7/10

In [61]:
model.fit(x_train_,y_train_,batch_size=100,epochs=1,validation_data=(x_test_,y_test_))

40/40 [==============================] - 171s 4s/step - loss: 0.0137 - categorical_accuracy: 0.9958 - val_loss: 0.0347 - val_categorical_accuracy: 0.9880


In [63]:
pred=model.predict(x_test_[0:500]).argmax(axis=1)
acc=tf.reduce_mean(tf.cast(tf.equal(pred,y_test[0:500]),tf.float32))
acc

<tf.Tensor: shape=(), dtype=float32, numpy=0.996>

In [1]:
from tensorflow.keras import layers, models, Model, Sequential


def VGG(feature, im_height=224, im_width=224, class_num=1000):
    # tensorflow中的tensor通道排序是NHWC
    input_image = layers.Input(shape=(im_height, im_width, 3), dtype="float32")
    x = feature(input_image)
    x = layers.Flatten()(x)
    x = layers.Dropout(rate=0.5)(x)
    x = layers.Dense(2048, activation='relu')(x)
    x = layers.Dropout(rate=0.5)(x)
    x = layers.Dense(2048, activation='relu')(x)
    x = layers.Dense(class_num)(x)
    output = layers.Softmax()(x)
    model = models.Model(inputs=input_image, outputs=output)
    return model


def features(cfg):
    feature_layers = []
    for v in cfg:
        if v == "M":
            feature_layers.append(layers.MaxPool2D(pool_size=2, strides=2))
        else:
            conv2d = layers.Conv2D(v, kernel_size=3, padding="SAME", activation="relu")
            feature_layers.append(conv2d)
    return Sequential(feature_layers, name="feature")


cfgs = {
    'vgg11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'vgg13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'vgg16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'vgg19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}


def vgg(model_name="vgg16", im_height=224, im_width=224, class_num=1000):
    try:
        cfg = cfgs[model_name]
    except:
        print("Warning: model number {} not in cfgs dict!".format(model_name))
        exit(-1)
    model = VGG(features(cfg), im_height=im_height, im_width=im_width, class_num=class_num)
    return model

vgg()

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

import tensorflow as tf
import json
import os
import glob


def main():
    data_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))  # get data root path
    image_path = os.path.join(data_root, "data_set", "flower_data")  # flower data set path
    train_dir = os.path.join(image_path, "train")
    validation_dir = os.path.join(image_path, "val")
    assert os.path.exists(train_dir), "cannot find {}".format(train_dir)
    assert os.path.exists(validation_dir), "cannot find {}".format(validation_dir)

    # create direction for saving weights
    if not os.path.exists("save_weights"):
        os.makedirs("save_weights")

    im_height = 224
    im_width = 224
    batch_size = 32
    epochs = 10

    _R_MEAN = 123.68
    _G_MEAN = 116.78
    _B_MEAN = 103.94

    def pre_function(img):
        # img = im.open('test.jpg')
        # img = np.array(img).astype(np.float32)
        img = img - [_R_MEAN, _G_MEAN, _B_MEAN]

        return img

    # data generator with data augmentation
    train_image_generator = ImageDataGenerator(horizontal_flip=True,
                                               preprocessing_function=pre_function)
    validation_image_generator = ImageDataGenerator(preprocessing_function=pre_function)

    train_data_gen = train_image_generator.flow_from_directory(directory=train_dir,
                                                               batch_size=batch_size,
                                                               shuffle=True,
                                                               target_size=(im_height, im_width),
                                                               class_mode='categorical')
    total_train = train_data_gen.n

    # get class dict
    class_indices = train_data_gen.class_indices

    # transform value and key of dict
    inverse_dict = dict((val, key) for key, val in class_indices.items())
    # write dict into json file
    json_str = json.dumps(inverse_dict, indent=4)
    with open('class_indices.json', 'w') as json_file:
        json_file.write(json_str)

    val_data_gen = validation_image_generator.flow_from_directory(directory=validation_dir,
                                                                  batch_size=batch_size,
                                                                  shuffle=False,
                                                                  target_size=(im_height, im_width),
                                                                  class_mode='categorical')
    total_val = val_data_gen.n
    print("using {} images for training, {} images for validation.".format(total_train,
                                                                           total_val))

    model = vgg("vgg16", 224, 224, 5)

    pre_weights_path = './pretrain_weights.ckpt'
    assert len(glob.glob(pre_weights_path+"*")), "cannot find {}".format(pre_weights_path)
    model.load_weights(pre_weights_path)
    for layer_t in model.layers:
        if layer_t.name == 'feature':
            layer_t.trainable = False
            break

    model.summary()

    # using keras high level api for training
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
                  metrics=["accuracy"])

    callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath='./save_weights/myAlex_{epoch}.h5',
                                                    save_best_only=True,
                                                    save_weights_only=True,
                                                    monitor='val_loss')]

    # tensorflow2.1 recommend to using fit
    history = model.fit(x=train_data_gen,
                        steps_per_epoch=total_train // batch_size,
                        epochs=epochs,
                        validation_data=val_data_gen,
                        validation_steps=total_val // batch_size,
                        callbacks=callbacks)


if __name__ == '__main__':
    main()


AssertionError: cannot find D:\jupyter\data_set\flower_data\train